<a href="https://colab.research.google.com/github/larissapoghosyan/Capstone_Project/blob/main/W2V_and_FastText_embeddings_both_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import numpy
import requests
from bs4 import BeautifulSoup
import re
import gensim
from gensim.models import Word2Vec,  FastText, Doc2Vec, KeyedVectors
from gensim import models
import gensim.downloader as api
import os, re, csv, math, codecs
import h5py
import nltk
from nltk import ngrams, sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer, LancasterStemmer, PorterStemmer
from nltk.corpus import stopwords
from sklearn import utils
from sys import getsizeof
nltk.download('punkt')

## Reading the Dataset

In [ ]:
# dataset = pd.read_csv('/content/drive/MyDrive/Lariba/Intent/outofscope-intent-classification-dataset.csv')
dataset = pd.read_csv('/content/IMDb_Reviews.csv',engine='python', error_bad_lines=False)
dataset = dataset.dropna()
print("Data Size: ", dataset.shape)
dataset.iloc[:, 0] = dataset.iloc[:,0].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Data Size:  (23700, 2)


## Get the w2v vectors

In [ ]:
wv_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
# wv_model = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary = True)

## Get Fast Text vectors

In [ ]:
# Downloading fasttext word vectors
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip
!ls

In [ ]:
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('/content/wiki-news-300d-1M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


999995it [01:08, 14679.71it/s]

found 999995 word vectors


## Extract FastText Embeddings

In [ ]:
new_dataset = []
not_in_vocab = []
dataset_vocab = []
for id, sent in enumerate(dataset.iloc[:,0]):
  new_sent = []
  
  for idx_, word in enumerate(sent):
    if word in embeddings_index.keys():
      new_sent.append(word)
    else:
      not_in_vocab.append(word)
    dataset_vocab.append(word)
  new_dataset.append(new_sent)

print(len(new_dataset)) # 50000, ok
new_dataset = np.array(new_dataset)


In [ ]:
data_vectors = np.zeros((len(new_dataset), 300))
for ix, data_point in enumerate(new_dataset):
    for word in data_point :
        data_vectors[ix, :]+=embeddings_index[word]
    data_vectors[ix, :]/=len(data_point)

In [ ]:
hf_ft = h5py.File('ft_vectors_intent.h5', 'w') # change intent to IMDb for IMDb datatset to avoid confusion
vec_lis_hf = hf_ft.create_dataset('vec_lis_hf', data = data_vectors)
hf_ft.close()

## Extract word2vec Embeddings

In [ ]:
new_dataset = []
not_in_vocab = []
dataset_vocab = []
for id, sent in enumerate(dataset.iloc[:,0]):
  new_sent = []
  
  for idx_, word in enumerate(sent):
    if word in wv_model.vocab:
      new_sent.append(word)
    else:
      not_in_vocab.append(word)
    dataset_vocab.append(word)
  #print(len(sent), len(new_sent))
  new_dataset.append(new_sent)

print(len(new_dataset)) # len in [50000, 23700] is ok
new_dataset = np.array(new_dataset)


In [ ]:
data_vectors = np.zeros((len(new_dataset), 300))
for ix, data_point in enumerate(new_dataset):
    for word in data_point :
        data_vectors[ix, :]+=wv_model[word]
    data_vectors[ix, :]/=len(data_point)
data_vectors.shape

In [ ]:
print('\n Number of words in the Dataset %s, \n Number of words not in wv vocabulary %s, \n Prc of words from Dataset not in the wv vpcabulary %s'
 % (len(dataset_vocab),
    len(not_in_vocab),
    (len(not_in_vocab)/len(dataset_vocab))*100)
 ) 
## (IMDB) 24% of words are not in the embeddings vector space
## (INTENT) 10.98% of words are not in the embeddings vector space


 Number of words in the Dataset 202053, 
 Number of words not in wv vocabulary 22186, 
 Prc of words from Dataset not in the wv vpcabulary 10.98028735034867


### Save the embeddings in h5py files:

In [ ]:
hf_wv = h5py.File('wv_vectors_intent.h5', 'w') # change intent to IMDb for IMDb datatset to avoid confusion
vec_lis_hf = hf_wv.create_dataset('vec_lis_hf', data = data_vectors)
hf_wv.close()